# Bounce

Inspired by a recent slew of posts on AI generating a Python program rendering a rotating square with a bouncing ball inside it, I wanted to revive some old knowledge, spice it up with additional information and teach myself to implement this adhering to some of my Pytonic principles.

## Step 1: Point, Vectors, Vertices and Polygons

Let's start with the very basics: a few classes to hold information for polygons and their vertices and have them drawn onto a canvas:

In [1]:
from dataclasses import dataclass, field
from typing import List

from ipycanvas import Canvas

@dataclass
class Vector:
    dx : float
    dy : float

    def __iter__(self):
        """
        allows for applying tuple()
        """
        yield self.dx
        yield self.dy

    def __truediv__(self, divider):
        return Vector(self.dx / divider, self.dy / divider)

@dataclass
class Point:
    x : float
    y : float

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def apply(self, transformation):
        (self.x, self.y, _) = transformation @ np.array([self.x, self.y, 1])
        return self
    
    def __iter__(self):
        """
        allows for applying tuple()
        """
        yield self.x
        yield self.y

class Vertex(Point):
    pass

@dataclass
class Polygon(List):
    vertices : List[Vertex] = field(default_factory=list)
    stroke   : str          = "000000"
    fill     : str          = None

    def render(self, canvas):
        if self.fill:
            canvas.fill_style   = f"#{self.fill}"
            canvas.fill_polygon(self)
        
        canvas.stroke_style = f"#{self.stroke}"
        canvas.stroke_polygon(self)

    def __iter__(self):
        """
        behave as a list, returning tuple versions of the vertices
        """
        for vertex in self.vertices:
            yield tuple(vertex)

canvas = Canvas(width=800, height=400, sync_image_data=True)

polygon1 = Polygon([ Vertex(100, 100), Vertex(150, 240), Vertex(50, 370) ])
polygon1.render(canvas)

polygon2 = Polygon([ Vertex(120, 120), Vertex(250,  50), Vertex(200, 270) ], stroke="00ff00")
polygon2.render(canvas)

polygon3 = Polygon([ Vertex(300, 100), Vertex(350, 240), Vertex(250, 370) ])
polygon3.render(canvas)

polygon4 = Polygon([ Vertex(295, 180), Vertex(425, 110), Vertex(375, 330) ], stroke="ff0000")
polygon4.render(canvas)

canvas

Canvas(height=400, sync_image_data=True, width=800)

I've drawn two situations of the same two polygons, one where they intersect and one where they don't. But I had to define them using explcit vertices. Wouldn't it be nicer if I could duplicate them and move them around? Let's introduce support for doing just that.

## Step 2: Transforming Polygons

To make this really useful, I basically want to `apply` transformations to polygons. To do so, I'll use transformation matrices, which makes this ultra genric and useful for a lot of things to come. To do so, the Polygon class is extended using a list of such transformations, which it applies on the fly when producing (x,y) coordinates.

E.g. to implement a translation by 200 along the X-axis, we use a transformation matrix:

```
 | 1 0 200 |   | x |
 | 0 1   0 | @ | y |
 | 0 0   1 |   | 1 |
```

In [2]:
from copy import deepcopy
import numpy as np

@dataclass
class Polygon(List):
    vertices        : List[Vertex]     = field(default_factory=list)
    stroke          : str              = "000000"
    fill            : str              = None
    transformations : List[np.ndarray] = field(default_factory=list)

    def render(self, canvas):
        if self.fill:
            canvas.fill_style   = f"#{self.fill}"
            canvas.fill_polygon(self)
        
        canvas.stroke_style = f"#{self.stroke}"
        canvas.stroke_polygon(self)

    def apply(self, transformation):
        """
        apply one transformation
        """
        self.vertices = [ vertex.apply(transformation) for vertex in self.vertices ]
        return self
    
    def transform(self, clear=True):
        """
        apply all transformations
        """
        for transformation in self.transformations:
           self.apply(transformation)
        # optionally clear when applied
        if clear:
            self.transformations.clear()
        return self

    def __iter__(self):
        """
        behave as a list, returning tuple versions of the vertices
        """
        for vertex in self.vertices:
            yield tuple(vertex)

canvas = Canvas(width=800, height=400, sync_image_data=True)

polygon1 = Polygon([ Vertex(100, 100), Vertex(150, 240),Vertex(50, 370) ])
polygon1.render(canvas)

polygon2 = Polygon([ Vertex(120, 120), Vertex(250, 50),Vertex(200, 270) ], stroke="00ff00")
polygon2.render(canvas)

def translate(dx=0, dy=0):
    return np.array([
        [ 1, 0, dx ],
        [ 0, 1, dy ],
        [ 0, 0, 1  ]
    ])

polygon3 = deepcopy(polygon1)
polygon3.transformations.append(translate(dx=200))
polygon3.transform().render(canvas)

polygon4 = deepcopy(polygon2)
polygon4.stroke = "ff0000"
polygon4.fill = "ff0000"
polygon4.transformations.append(translate(dx=175))
polygon4.transformations.append(translate(dy=60))
polygon4.transform().render(canvas)

canvas

Canvas(height=400, sync_image_data=True, width=800)

Let's have some fun and throw some rotation in the mix. A rotation around the origin is defined by a transformation matrix with the following structure:

```
 | cos a  -sin a 0 |
 | sin x   cos a 0 |
 | 0       0     1 |
```

And combining this with the translation, we can implement a rotation around an arbitrary point:

In [3]:
import math

def rotate(a=0):
    return np.array([
        [ math.cos(a), -math.sin(a), 0 ],
        [ math.sin(a), math.cos(a),  0 ],
        [ 0,           0,            1 ]
    ])

canvas = Canvas(width=800, height=400, sync_image_data=True)

polygon1.render(canvas)
polygon2.render(canvas)

def rotate_around(a, x, y):
    return translate(dx=x, dy=y) @ rotate(math.radians(a)) @ translate(dx=-x, dy=-y)

print(rotate_around(20, 325, 210))

polygon3.transformations.append(rotate_around(20, 325, 210))
polygon3.transform().render(canvas)

polygon4.transformations.append(rotate_around(20, 325, 210))
polygon4.transform().render(canvas)

canvas

[[  0.93969262  -0.34202014  91.42412834]
 [  0.34202014   0.93969262 -98.49199695]
 [  0.           0.           1.        ]]


Canvas(height=400, sync_image_data=True, width=800)

## Step 3: Animating a Box

With polygons and rotation we have everything needed in place to create a rotating box. To hold all objects and drive the animation, I'm introducing a small World class. I also introduce a Box function, generating a Polygon from left, top, width and height properties.

In [4]:
from ipycanvas import hold_canvas
import time

def Box(left, top, width, height):
    return Polygon([ Vertex(left, top), Vertex(left+width, top), Vertex(left+width, top+height), Vertex(left, top+height) ])

@dataclass
class Shape:
    polygon    : Polygon
    animations : List[np.ndarray] = field(default_factory=list)

    def move(self):
        for animation in self.animations:
            self.polygon.apply(animation)

    def render(self, canvas):
        self.polygon.render(canvas)

@dataclass
class World:
    width  : int         = 800
    height : int         = 500
    shapes : List[Shape] = field(default_factory=list)
    canvas : Canvas      = field(init=False, repr=False, default=None)

    def __post_init__(self):
        self.canvas = Canvas(width=self.width, height=self.height, sync_image_data=True)
    
    def _move(self):
        for shape in self.shapes:
            shape.move()
    
    def _render(self):
        self.canvas.stroke_rect(0, 0, self.width, self.height)
        for shape in self.shapes:
            shape.render(self.canvas)

    def draw(self, iterations=1):
        for _ in range(iterations):
            self._render()
            self._move()
        return self.canvas

    def animate(self):
        display(self.canvas)
        while True:
            with hold_canvas():
                self.canvas.clear()
                self._render()
                self._move()
            time.sleep(0.005)

world = World()

box = Shape(Box(150, 150, 200, 200))
box.animations.append(rotate_around(5, 250, 250))

world.shapes.append(box)
world.draw(5)
# world.animate()


Canvas(sync_image_data=True, width=800)

## Step 4: Collision Detection

If we want to have a bouncing ball inside the box, we need to detect when it hits a wall and change its course. In case of basic boxes (e.g. without rotations applied to them) it is pretty easy, yet once rotated, a little more elaborate algorithm is needed.

As a strategy, we'll consider that a shape is (still) inside a polygon, if all of its points are inside that polygon. This holds for convex polygons. Maybe later we can expand this to concave polygons too.

Later we might also introduce other bouncing shapes. Those can collide into each other. To detect such collisions, we need another algorithm: SAT - the Separating Axis Theorem. This theorem states that two convex polygons are not overlapping if one can draw a line between them. So, given an algorithm that checks if such line exists, we can determine if two convex polygons overlap.

### PIP - Point in Polygon

To determine if all points of a polygon are inside another (convex) polygon we need a point in polygon algorithm (PIP). This algorithm relies on the fact that a "ray" from outside the polygon to a point will intersect a polygon an even number of times when the point lies outside the polygon and an odd number of times when it lies inside.

Let's build towards that step by step.

#### Formula for a Line given two points

Given two points `(x1, y1) and (x2, y2)`, the line running throught those points is defined by the formula `y =  mx + c`. `m = ((y2 - y1) / (x2 - x1))` and `c` can be computed given one point as `c = y - mx`.

In [5]:
@dataclass
class Line:
    p1     : Vertex
    p2     : Vertex
    stroke : str    = "000000"

    @property
    def vertical(self):
        return self.p1.x == self.p2.x
    
    @property
    def m(self):
        v = self.p2 - self.p1
        return v.dy / v.dx

    @property
    def c(self):
        return self.p1.y - self.m * self.p1.x

    def __str__(self):
        if self.vertical:
            return f"x = {self.x}"
        else:
            return f"y = {self.m}x + {self.c}"

    def render(self, canvas):
        canvas.stroke_style = f"#{self.stroke}"
        if self.vertical:
            canvas.stroke_line(self.p1.x, 0, self.p1.x, canvas.height)
        else:
            canvas.stroke_line(0, self(0), canvas.width, self(canvas.width))
    
    @property
    def f(self):
        return lambda x : self.m * x + self.c   
    
    def __call__(self, x):
        return self.f(x)

    def intersection(self, other):
        if self.vertical and other.vertical:
            return None
        elif self.vertical:
            return Point(self.p1.x, other(self.p1.x))
        elif other.vertical:
            return Point(other.p1.x, self(other.p1.x))
        else:
            x = (other.c - self.c) / (self.m - other.m)
            return Point(x, self(x))
    
    def intersects(self, line):
        self(line.p1.x) == line.p1.y

class Segment(Line):
    def __str__(self):
        return f"{super().__str__()} | x in [{self.p1.x}, {self.p2.x}]"

    def render(self, canvas):
        canvas.stroke_style = f"#{self.stroke}"
        canvas.stroke_line(self.p1.x, self.p1.y, self.p2.x, self.p2.y)
    
    def __call__(self, x):
        if (x < min(self.p1.x, self.p2.x)) or (max(self.p1.x, self.p2.x) < x):
            return None
        y = super().__call__(x)
        if (y < min(self.p1.y, self.p2.y)) or (max(self.p1.y, self.p2.y) < y):
            return None
        return y

    def intersection(self, other):
        p = super().intersection(other)
        if not p.x or not p.y:
            return None
        if (p.x < min(self.p1.x, self.p2.x)) or (max(self.p1.x, self.p2.x) < p.x):
            return None
        if (p.y < min(self.p1.y, self.p2.y)) or (max(self.p1.y, self.p2.y) < p.y):
            return None
        return p

p1 = Vertex(4, 5)
p2 = Vertex(8, 8)

# let's see if (6, 6.5) is on a line through p1 and p2
p3 = Vertex(3, 4.25)

line = Line(p1, p2)
print(line)
print(line(p3.x) == p3.y)

# does it also lie on the segment through p1 and p2
segment = Segment(p1, p2)
print(segment)
print(segment(p3.x) == p3.y)

y = 0.75x + 2.0
True
y = 0.75x + 2.0 | x in [4, 8]
False


Given the math above, we can determine if a point is on a line segment, or an edge for that matter. So how can we use that to check if a ray through a point intersects with that edge?

We can take, for example, the vertical line through the point, the intersection point of that line and the edge and check the if that intersection is within the Y boundaries of the segment:

In [6]:
canvas = Canvas(width=800, height=400, sync_image_data=True)

polygon1 = Polygon([ Vertex(100, 100), Vertex(150, 240),Vertex(50, 370) ])
polygon1.render(canvas)

polygon2 = Polygon([ Vertex(120, 120), Vertex(250, 50),Vertex(200, 270) ])
polygon2.render(canvas)

@dataclass
class Circle:
    center : Point
    radius : float
    stroke          : str              = "000000"
    fill            : str              = None

    def render(self, canvas):
        if self.fill:
            canvas.fill_style   = f"#{self.fill}"
            canvas.fill_circle(self.center.x, self.center.y, self.radius)

        if self.stroke:
            canvas.stroke_style = f"#{self.stroke}"
            canvas.stroke_circle(self.center.x, self.center.y, self.radius)

p = Point(130, 225)

vertical = Segment(Vertex(p.x, 0), p, stroke="0000ff")
vertical.render(canvas)

Circle(p, 5, fill="ffcc00", stroke=None).render(canvas)

def edges(polygon):
    return [
        Segment(v1, v2)
        for v1, v2 in list(zip(polygon.vertices, polygon.vertices[1:])) + [(polygon.vertices[-1], polygon.vertices[0])]
    ]

def intersections(polygon, segment):
    for edge in edges(polygon):
        intersection = segment.intersection(edge)
        if intersection:
            yield intersection

[ Circle(p, 5, fill="00ff00", stroke=None).render(canvas) for p in intersections(polygon1, vertical) ]
[ Circle(p, 5, fill="ff0000", stroke=None).render(canvas) for p in intersections(polygon2, vertical) ]

canvas

Canvas(height=400, sync_image_data=True, width=800)

### SAT - Separating Axis Theorem

Given two points: `p1 = (4,5)` and `p2 = (8,8)`, vector `v = p2 - p1 = (4,3)`. The perpendicular vector is then `vp = (-3,4)`. The length or magnitude of the vector is determined by `sqrt(-3^2 + 4^2) = 5`. Normalizing the vector is done by dividing its components by its magnitude: `vpn = (-3/5, 4/5)`. Finally, given two vectors, we can project one onto the other using the dot product: `A . B = Sum(AiBi)`. So given `ex = (1, 0)` the dot product `vpn . ex = -3/5 + 0 = -3/5 = -0,6`.

In [13]:
p1 = Point(4, 5)
p2 = Point(8, 8)
v  = p2 - p1

def perpendicular(v):
    return Vector(v.dy * -1, v.dx)

vp = perpendicular(v)

def normalize(v):
    norm = np.linalg.norm(tuple(v))
    if norm == 0: 
       return v
    return v / norm

vpn = normalize(vp)

ex = np.array([1,0])

np.dot(tuple(vpn), ex)

-0.6

Construct a list of all normalized vectors perpendicular to all edges and project all vertices of each polygon onto those vectors, keeping the minimum and maximum and checking if the projections overlap. If not, there is a separating axis and they don't overlap.

In [8]:
print(edges(polygon1))
print(edges(polygon2))

[Segment(p1=Vertex(x=100, y=100), p2=Vertex(x=150, y=240), stroke='000000'), Segment(p1=Vertex(x=150, y=240), p2=Vertex(x=50, y=370), stroke='000000'), Segment(p1=Vertex(x=50, y=370), p2=Vertex(x=100, y=100), stroke='000000')]
[Segment(p1=Vertex(x=120, y=120), p2=Vertex(x=250, y=50), stroke='000000'), Segment(p1=Vertex(x=250, y=50), p2=Vertex(x=200, y=270), stroke='000000'), Segment(p1=Vertex(x=200, y=270), p2=Vertex(x=120, y=120), stroke='000000')]


In [15]:
def perpendicular_edge_vectors(polygon):
    return [ normalize(perpendicular(edge.p1 - edge.p2)) for edge in edges(polygon) ]

pevs = perpendicular_edge_vectors(polygon1) + perpendicular_edge_vectors(polygon2)
pevs

[Vector(dx=0.9417419115948374, dy=-0.33633639699815626),
 Vector(dx=0.7926239891046001, dy=0.6097107608496924),
 Vector(dx=-0.9832820049844602, dy=-0.18208926018230745),
 Vector(dx=-0.47409982303501746, dy=-0.8804710999221753),
 Vector(dx=0.9751328557914597, dy=0.2216211035889681),
 Vector(dx=-0.8823529411764706, dy=0.47058823529411764)]

In [17]:
def project(polygon, pev):
    projections = [ np.dot(tuple(vertex), tuple(pev)) for vertex in polygon.vertices ]
    return ( min(projections), max(projections) )

[ (project(polygon1, pev), project(polygon2, pev)) for pev in pevs ]

[((-77.35737130957595, 60.54055145966812),
  (72.64866175160174, 218.61865804880156)),
 ((140.23347499542925, 265.2241809696162),
  (168.2801699945151, 323.146703250337)),
 ((-191.1937231914228, -116.53712651667676),
  (-254.9249642552304, -139.84455182001213)),
 ((-349.47929812295575, -135.4570922957193),
  (-332.5471615859908, -162.54851075486312)),
 ((119.67539593804278, 199.4589932300713),
  (143.61047512565133, 254.86426912731335)),
 ((-41.1764705882353, 130.0), (-197.05882352941177, -49.41176470588235))]

In [18]:
def projections_overlap(p1, p2):
    """
      p1    - .......... +
      p2           - .......... +
      overlapping if p1_min in p2 or p2_min in p1
    """
    return ( p1[0] < p2[1] and p1[0] > p2[0] ) or ( p2[0] < p1[1] and p2[0] > p1[0] )

print(projections_overlap( project(polygon1, pevs[0]), project(polygon2, pevs[0]) ))
print(projections_overlap( project(polygon1, pevs[4]), project(polygon2, pevs[4]) ))

False
True


In [19]:
def polygons_overlap(poly1, poly2):
    """
    simple implementation computing all overlaps, while we could stop as soon as we found 1
    """
    pevs = perpendicular_edge_vectors(poly1) + perpendicular_edge_vectors(poly2)
    return [ projections_overlap( project(poly1, pev), project(poly2, pev) ) for pev in pevs ]

print(polygons_overlap(polygon1, polygon2))
print(polygons_overlap(polygon3, polygon4))

[False, True, True, True, True, False]
[True, True, True, True, True, True]
